# Объединение данных, данные о времени

## Постановка задачи

В папке `subsid` содержатся файлы с продажами продуктов через телемаркетинг (`tm_sales_1`, `tm_sales_2`, ...). Каждый файл содержит, как минимум, 4 колонки (поля): `FILIAL_ID`, `SUBS_ID`, `PROD_ID`, `ACT_DTTM`.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

Особенности данных:

- сотрудники телемаркетинга не всегда указывают полный `id`, если `id` нет в начале `SUBS_ID`, то нужно его добавить
- поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
- продажа не засчитывается, если отключение (`END_DTTM`) произошло меньше чем через 5 минут после подключения (`START_DTTM`)
- если в файле с продажами встречается строка без указанного `SUBS_ID`, она пропускается

## Импорт необходимых библиотек и файлов с данными

In [1]:
import pandas as pd
import os
import datetime as dt

In [3]:
data_dir_name = './subsid/'
filenames = os.listdir('./subsid/')
logs_file = filenames[0]
sales_files = filenames[1:]
print(filenames)

['prod_activations_logs.csv', 'tm_sales_1.csv', 'tm_sales_2.csv', 'tm_sales_3.csv']


In [4]:
df_sales = None
for filename in sales_files:
    filepath = data_dir_name + filename
    df = pd.read_csv(filepath,
                     sep = ';',
                     parse_dates = ['ACT_DTTM'],
                     date_parser = lambda x: pd.to_datetime(x, format = "%d-%m-%Y %H:%M"))
    df = df[['FILIAL_ID', 'SUBS_ID', 'PROD_ID', 'ACT_DTTM']]
    df = df.dropna(subset = ['SUBS_ID'])
    df['SUBS_ID'] = [i if i.startswith('id') else ('id' + i) for i in df['SUBS_ID']]

    if (df_sales is None) :
        df_sales = df
    else :
        df_sales = df_sales.append(df, ignore_index = True)

In [5]:
df_sales.shape

(9, 4)

In [6]:
df_sales.dtypes

FILIAL_ID             int64
SUBS_ID              object
PROD_ID               int64
ACT_DTTM     datetime64[ns]
dtype: object

In [7]:
df_sales

,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM
0,1,id4651830,1954,2020-03-20 14:59:00
1,5,id7646509,6431,2020-03-19 13:00:00
2,4,id7412683,3313,2020-03-22 17:25:00
3,3,id5416547,1743,2020-03-17 10:17:00
4,7,id8362218,9879,2020-03-05 11:42:00
5,2,id2185490,3210,2020-03-16 16:28:00
6,3,id5764122,1499,2020-03-18 15:44:00
7,6,id7642700,3020,2020-03-15 14:21:00
8,2,id1374509,5677,2020-03-17 11:48:00


In [8]:
df_logs = pd.read_csv(data_dir_name + logs_file,
                      sep = ';')
df_logs['START_DTTM'] = pd.to_datetime(df_logs['START_DTTM'],
                                       format = "%d-%m-%Y %H:%M")
df_logs['END_DTTM'] = pd.to_datetime(df_logs['END_DTTM'],
                                       format = "%d-%m-%Y %H:%M")
# parse_dates = ['START_DTTM', 'END_DTTM'],
# date_parser = lambda x: pd.to_datetime(x, format = "%d-%m-%Y %H:%M")
df_logs = df_logs.reset_index(drop = True)
df_logs['PROD_ID'] = df_logs['PROD_ID'].astype('int64')

In [9]:
df_logs

,SUBS_ID,PROD_ID,START_DTTM,END_DTTM
0,id4651830,1954,2020-03-20 14:59:00,2020-12-01 00:00:00
1,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:03:00
2,id7461794,3310,2020-03-20 17:25:00,2020-12-01 00:00:00
3,id5416547,1743,2020-03-17 10:17:00,2020-03-25 11:00:00
4,id8238421,1859,2020-03-01 11:42:00,2020-03-01 11:43:00
5,id8641743,2752,2020-03-20 15:44:00,2020-04-21 15:44:00
6,id2185490,3210,2020-03-16 16:28:00,2020-12-01 00:00:00
7,id7642700,3020,2020-03-15 14:21:00,2020-03-15 23:42:00
8,id8741631,5677,2020-03-19 12:28:00,2020-12-01 00:00:00


In [10]:
df_logs.shape

(9, 4)

In [11]:
df_logs.dtypes

SUBS_ID               object
PROD_ID                int64
START_DTTM    datetime64[ns]
END_DTTM      datetime64[ns]
dtype: object

In [12]:
df_summary = \
    df_sales.merge(df_logs,
                   how = 'inner',
                   left_on = ['SUBS_ID', 'PROD_ID'],
                   right_on = ['SUBS_ID', 'PROD_ID'])

In [13]:
df_summary['difference'] = df_summary['END_DTTM'] - df_summary['START_DTTM']

In [14]:
df_summary

,FILIAL_ID,SUBS_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,difference
0,1,id4651830,1954,2020-03-20 14:59:00,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,5,id7646509,6431,2020-03-19 13:00:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,3,id5416547,1743,2020-03-17 10:17:00,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,2,id2185490,3210,2020-03-16 16:28:00,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,6,id7642700,3020,2020-03-15 14:21:00,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [15]:
df_summary = \
    df_summary[df_summary.difference >= '5m'] \
    .sort_values(by = ['SUBS_ID'])     \
    .reset_index(drop = True)

In [16]:
df_summary['SUBS_ID'].to_list()

['id2185490', 'id4651830', 'id5416547', 'id7642700']

In [17]:
df_summary_to_file = df_summary[['ACT_DTTM', 'FILIAL_ID', 'difference']]
df_summary_to_file.to_csv('./sales_logs.csv', 
                  sep = ';')